# Stimulated emission protocol with BD followed by repeated π (n+1 and n+1) 

In [1]:
from pylab import*
import matplotlib.pyplot as plt
import seaborn as sns
from h5py import File
import pandas as pd
from slab.dsfit import*
import json
import numpy as np
from datetime import datetime
import scipy as sc
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
# from qutip import *
from h5py import File
import os
from slab.dataanalysis import get_next_filename

Warning could not load Chase AWG dll, check that dll located at 'C:\_Lib\python\slab\instruments\awg\chase\dax22000_lib_DLL32.dll'
Could not load InstrumentManagerWindow
Warning could not load LDA labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\VNX_atten.dll'
Warning could not load LMS labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\vnx_fmsynth.dll'
Warning could not load LPS labbrick dll, check that dll located at 'C:\_Lib\python\slab\instruments\labbrick\VNX_dps.dll'
Could not load labbrick


C:\ProgramData\Miniconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "
C:\ProgramData\Miniconda3\lib\site-packages\visa.py:13: FutureWarning: The visa module provided by PyVISA is being deprecated. You can replace `import visa` by `import pyvisa as visa` to achieve the same effect.

The reason for the deprecation is the possible conflict with the visa package provided by the https://github.com/visa-sdk/visa-python which can result in hard to debug situations.
  warnings.warn(


# Fit functions

In [3]:
def coherent_state(n, alpha):
    return np.exp(-abs(alpha)**2)*abs(alpha)**(2*n)/scipy.special.factorial(n)

def line(x, m, b):
    return m*x+b

def proportional(x, m):
    return m * x

def gfromchi(chi,alpha,delta):
    return np.sqrt(chi*delta*(delta+alpha)/alpha)

def gaussfuncsum(p, x):
    """p[0]+p[1]/(1+(x-p[2])**2/p[3]**2)"""
    y = 0
    for ii in range(N):
        y = y + p[3*ii+1]*exp(-(x-(p[3*ii+2]))**2/2/p[3*ii+3]**2)
    return y

def gaussfuncsum_with_baseline(x, *args):
    """p[0]+p[1]/(1+(x-p[2])**2/p[3]**2)"""
    y = 0 
    p = args
#     print(len(p))
    for ii in range(7):
        y = y + p[3*ii+1]*exp(-(x-(p[3*ii+2]))**2/2/p[3*ii+3]**2)
    y += p[0]
    return y
def fitgausssum(xdata, ydata, fitparams=None, domain=None, showfit=False,
                showstartfit=False, label="", debug=False):
    """fit lorentzian:
        returns [offset,amplitude,center,hwhm]"""
    if domain is not None:
        fitdatax,fitdatay = selectdomain(xdata, ydata, domain)
    else:
        fitdatax = xdata
        fitdatay = ydata
    
    if fitparams is None:
        fitparams = 0*ones(3*N+1)
        fitparams[0] = (fitdatay[0] + fitdatay[-1])/2.
        fitparams[1] = max(fitdatay) - min(fitdatay)
        fitparams[2] = fitdatax[np.argmax(fitdatay)]
        fitparams[3] = (max(fitdatax) - min(fitdatax))/10.
    
    if debug==True: 
        print(fitparams)
        
    p1 = fitgeneral(fitdatax, fitdatay, gaussfuncsum, fitparams, domain=None, showfit=showfit, 
                    showstartfit=showstartfit, label=label)
    p1[3]=abs(p1[3])
    
    return p1

def fitgausssum_with_baseline(xdata, ydata, fitparams=None):
    """fit lorentzian:
        returns [offset,amplitude,center,hwhm]"""
    fitdatax = xdata
    fitdatay = ydata
#     if fitparams is None:
#         fitparams = 0*ones(3*N+1)
#         fitparams[0] = (fitdatay[0]+fitdatay[-1])/2.
#         fitparams[1] = max(fitdatay)-min(fitdatay)
#         fitparams[2] = fitdatax[np.argmax(fitdatay)]
#         fitparams[3] = (max(fitdatax)-min(fitdatax))/10.
    
#     p1 = fitgeneral(fitdatax, fitdatay, gaussfuncsum_with_baseline, fitparams, domain=None, showfit=showfit,
#                     showstartfit=showstartfit, label=label)
#     bounds=(0, [3., 1., 0.5])
        
    popt, pcov = curve_fit(gaussfuncsum_with_baseline, fitdatax, fitdatay, p0=fitparams, bounds=(0, np.inf))

    return popt, pcov

def fitamp(p1):
    fitamparray = []
    fitfreqarray = []
    n_peaks = int((len(p1)-1)/3)
    for i in range(n_peaks):
        fitamparray.append(p1[3*i+1])
        fitfreqarray.append(p1[3*i+2])
    return fitamparray , fitfreqarray

def fitcoherentstate(peak_val):
    xdata = np.arange(len(peak_val))
    ydata = peak_val
    popt, pcov = curve_fit(coherent_state, xdata, ydata)
    return popt[0], np.sqrt(pcov[0][0])

def expfunc2(x, p):
    """p[0]+p[1]*exp(-(x-p[2])/p[3])"""
    return p[0]*np.exp**(-(x-p[1])/p[2])

def fitexp2(xdata,ydata,fitparams=None,domain=None,showfit=False,showstartfit=False,label=""):
    """Fit exponential decay (p[0]*exp(-(x-p[1])/p[2]))"""
    if domain is not None:
        fitdatax,fitdatay = selectdomain(xdata,ydata,domain)
    else:
        fitdatax=xdata
        fitdatay=ydata
    if fitparams is None:    
        fitparams=[0.,0.,0.,0.]
        fitparams[0]=fitdatay[0]-fitdatay[-1]
        fitparams[1]=fitdatax[0]
        fitparams[2]=(fitdatax[-1]-fitdatax[0])/5.
    #print fitparams
    p1 = fitgeneral(fitdatax, fitdatay, expfunc2, fitparams, domain=None, showfit=showfit, showstartfit=showstartfit,
                    label=label)
    return p1   

def expfunc_test(x, a, b, c):
    return b*np.exp(-(x-a)/c)

def expfunc_baseline(x, a, b, c):
    return a*np.exp(-x/b) + c

def doublegauss(bins, *p):
    a1, sigma1, mu1 = p[0], p[1], p[2]
    a2, sigma2, mu2 = p[3], p[4], p[5]

    y1 = a1*((1 / (np.sqrt(2 * np.pi) * sigma1)) *
     np.exp(-0.5 * (1 / sigma1 * (bins - mu1))**2))
    y2 = a2*((1 / (np.sqrt(2 * np.pi) * sigma2)) *
     np.exp(-0.5 * (1 / sigma2 * (bins - mu2))**2))
    y = y1+y2
    
    return y
def gaussian2d(x, y, x0, y0, xalpha, yalpha, A):
    return A * np.exp( -((x-x0)/xalpha)**2 -((y-y0)/yalpha)**2)


def hist(filename=None, data=None, plot=True, ran=1.0):
    
    if data == None:
        with File(filename,'r') as a:
            ig = array(a['ig'])
            qg = array(a['qg'])
            ie = array(a['ie'])
            qe = array(a['qe'])
            a.close()
    else:
        ig = data[0]
        qg = data[1]
        ie = data[2]
        qe = data[3]

    numbins = 200
    
    xg, yg = np.median(ig), np.median(qg)
    xe, ye = np.median(ie), np.median(qe)

    if plot==True:
        fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))
        fig.tight_layout()

        axs[0].scatter(ig, qg, label='g', color='b', marker='*')
        axs[0].scatter(ie, qe, label='e', color='r', marker='*')
        axs[0].scatter(xg, yg, color='k', marker='o')
        axs[0].scatter(xe, ye, color='k', marker='o')
        axs[0].set_xlabel('I (a.u.)')
        axs[0].set_ylabel('Q (a.u.)')
        axs[0].legend(loc='upper right')
        axs[0].set_title('Unrotated')
        axs[0].axis('equal')
    """Compute the rotation angle"""
    theta = -arctan((ye-yg)/(xe-xg))
    """Rotate the IQ data"""
    ig_new = ig*cos(theta) - qg*sin(theta)
    qg_new = ig*sin(theta) + qg*cos(theta) 
    ie_new = ie*cos(theta) - qe*sin(theta)
    qe_new = ie*sin(theta) + qe*cos(theta)
    
    """New means of each blob"""
    xg, yg = np.median(ig_new), np.median(qg_new)
    xe, ye = np.median(ie_new), np.median(qe_new)

    xlims = [xg-ran, xg+ran]
    ylims = [yg-ran, yg+ran]

    if plot==True:
        axs[1].scatter(ig_new, qg_new, label='g', color='b', marker='*')
        axs[1].scatter(ie_new, qe_new, label='e', color='r', marker='*')
        axs[1].scatter(xg, yg, color='k', marker='o')
        axs[1].scatter(xe, ye, color='k', marker='o')    
        axs[1].set_xlabel('I (a.u.)')
        axs[1].legend(loc='upper right')
        axs[1].set_title('Rotated')
        axs[1].axis('equal')

        """X and Y ranges for histogram"""
        
        ng, binsg, pg = axs[2].hist(ig_new, bins=numbins, range = xlims, color='b', label='g', alpha=0.5)
    #     popt, pcov = curve_fit(doublegauss, xdata=binsg[:-1], ydata=ng, p0=p0)
    #     mu_g = popt[2]
    #     axs[2].plot(binsg, doublegauss(binsg, *popt), 'k--', linewidth=2 )
        ne, binse, pe = axs[2].hist(ie_new, bins=numbins, range = xlims, color='r', label='e', alpha=0.5)
    #     popt, pcov = curve_fit(doublegauss, xdata=binse[:-1], ydata=ne, p0=p0)
    #     mu_e = popt[5]
    #     axs[2].plot(binse, doublegauss(binse, *popt), 'k--', linewidth=2 )
    #     axs[2].text(0.5*(mu_g + mu_e), 0.3*np.max(ne), "$\mu_{g}$ = %.4f \n $\mu_{e}$ = %.4f"%(mu_g, mu_e), fontsize=16)

        axs[2].set_xlabel('I(a.u.)')        
        fig.show()
        
    else:        
        ng, binsg = np.histogram(ig_new, bins=numbins, range = xlims)
        ne, binse = np.histogram(ie_new, bins=numbins, range = xlims)

    """Compute the fidelity using overlap of the histograms"""
    fid = np.abs(((np.cumsum(ng) - np.cumsum(ne)) / (0.5*ng.sum() + 0.5*ne.sum()))).max()

    return fid, theta

def rot_data(i, q, hist_filename=None):
    
    fid, theta = hist(hist_filename, ran=0.1)
    print(fid, theta)
    """Rotate the IQ data"""
    i_new = i*cos(theta) - q*sin(theta)
    q_new = i*sin(theta) + q*cos(theta) 
    
    return (i_new, q_new)


def ramsfit(tR, n0, phi0, a):
    T2 = 120e-6
    dephase = 1/T2
    detune = 2*np.pi*1e6 
    chi = 2*np.pi*380e3
    kappa = 2*np.pi*(8.0517e9)/8800
    tau = []
    z = []
    res = []
    for i,t in enumerate(tR):
        tau.append((1-np.exp(-complex(kappa*tR[i], 2*chi*tR[i])))/complex(kappa, 2*chi))
        z.append(np.exp(complex(-dephase*tR[i] , phi0 - detune*tR[i] - 2*n0*chi*tau[i])))
        res.append(a*0.5*(1-np.imag(z[i])))
    return np.array(res)

def cav_response_new(p, x):
    """(p[0]/p[1])/(-1/2*p[0]/p[1] - 1j*(x-p[0])"""
    ### p[0]=center freq, p[1]=kappa
    temp = (p[1])/(p[1] - 1j*(x-p[0]))
    return temp/max(abs(temp))

def IF_window(p,x):
    ### p[0] = center freq, p[1] = window width
    temp = zeros(len(x)) + 1j*zeros(len(x))
    for ii in range(len(x)):
        if x[ii]>(p[0]-p[1]) and x[ii]<(p[0]+p[1]):
            temp[ii] = 1/sqrt(2)*(1+1j)
        else:
            pass
    return temp/max(abs(temp))

def erf_t(A, sig, tc, tb, t):
    #A-Amplitude, sig-Gaussian Filter Width, tc-Core Pulse length, tb - zero-amplitude buffer length
    return (A/2)*(sc.special.erf((t-tb)/(sqrt(2)*sig))-sc.special.erf((t-tc-tb)/(sqrt(2)*sig)))

# HMM Model

In [23]:
class hmm_analysis:

    def __init__(self, qubit_params = None, cavity_params = None, readout_params = None):
        
        self.qubit_params = qubit_params
        self.cavity_params = cavity_params
        self.readout_params = readout_params

        """All the timescales are in μs"""
        self.qubit_t1 = qubit_params['t1']
        self.qubit_t2 = qubit_params['t2']
        self.qubit_nth = qubit_params['nth']
        
        self.cavity_t1 = cavity_params['t1']
        self.cavity_nth = cavity_params['nth']
        
        self.readout_len = readout_params['length']        
        self.trigger_period = readout_params['trigger']
        self.pi_length = readout_params['pi_pulse']

    ##----------------------------------------------------------------##
    def forward(self, meas_seq, T, E):
        num_meas = len(meas_seq)
        N = T.shape[0]
        alpha = zeros((num_meas, N))
        pi = [0.25, 0.25, 0.25, 0.25]
        alpha[0] = pi*E[:,meas_seq[0]]
        for t in range(1, num_meas):
            alpha[t] = alpha[t-1].dot(T) * E[:, meas_seq[t]]
        return alpha

    def backward(self, meas_seq, T, E):
        N = T.shape[0]
        num_meas = len(meas_seq)
        beta = zeros((N,num_meas))
        beta[:,-1:] = 1
        for t in reversed(range(num_meas-1)):
            for n in range(N):
                beta[n,t] = sum(beta[:,t+1] * T[n,:] * E[:, meas_seq[t+1]])
        return beta

    def likelihood(self, meas_seq, T, E):
        # returns log P(Y  \mid  model)
        # using the forward part of the forward-backward algorithm
        return  self.forward(meas_seq, T, E)[-1].sum()

    def gamma(self, meas_seq, T, E):
        alpha = self.forward(meas_seq, T, E)
        beta  = self.backward(meas_seq, T, E)
        obs_prob = self.likelihood(meas_seq, T, E)
        return (multiply(alpha, beta.T) / obs_prob)

    def viterbi(self, meas_seq, T, E):
        # returns the most likely state sequence given observed sequence x
        # using the Viterbi algorithm
        num_meas = len(meas_seq)
        N = T.shape[0]
        delta = zeros((num_meas, N))
        psi = zeros((num_meas, N))
        pi = [0.25,0.25,0.25,0.25]
        delta[0] = pi*E[:,meas_seq[0]]
        for t in range(1, num_meas):
            for j in range(N):
                delta[t,j] = max(delta[t-1]*T[:,j]) * E[j, meas_seq[t]]
                psi[t,j] = argmax(delta[t-1]*T[:,j])

        # backtrack
        states = zeros(num_meas, dtype=int32)
        states[num_meas-1] = argmax(delta[num_meas-1])
        for t in range(num_meas-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
        return states
    ##----------------------------------------------------------------##
    def alpha_awg_cal(self, cav_amp=0.4, cav_len=250):
        # takes input array of amps and length and converts them to output array of alphas,
        # using a calibration h5 file defined in the experiment config
        # pull calibration data from file, handling properly in case of multimode cavity
        cal_path = 'C:\\_Lib\\python\\slab\\experiments\\qm_opx\\drive_calibration'

        fn_file = cal_path + '\\00000_2021_7_30_cavity_square.h5'

        with File(fn_file, 'r') as f:
            omegas = np.array(f['omegas'])
            amps = np.array(f['amps'])

        # assume zero frequency at zero amplitude, used for interpolation function
        omegas = np.append(omegas, 0.0)
        amps = np.append(amps, 0.0)

        o_s = omegas
        a_s = amps

        # interpolate data, transfer_fn is a function that for each amp returns the corresponding omega
        transfer_fn = scipy.interpolate.interp1d(a_s, o_s)

        omega_desired = transfer_fn(cav_amp)
        alpha = omega_desired * cav_len

        """Returns alpha in the cavity"""
        return alpha
        
    ##----------------------------------------------------------------##
    def openfile(self, filename):
        
        return File(filename,'r')

    def stateprep(self, data_filename, fock_state =0,  at_end=True):
        
        """Readout fidelities from an independent measurement"""
        g_infidelity, e_infidelity = 0.0246, 0.0408
        
        self.a = self.openfile(data_filename)

        num = pd.DataFrame(self.a['num'])[:]
        bit = pd.DataFrame(self.a['bit'])[:]
            
        cav_amp = np.array(self.a['amp'])
        cav_len = np.array(self.a['time'])
        npi_m = int(np.array(self.a['pi_m']))
        npi_n = int(np.array(self.a['pi_n']))

        self.a.close()
        
        df = bit
                
        alpha = self.alpha_awg_cal(cav_amp, cav_len)
        print('# of π at m= {}, at n = {}'.format(npi_m, npi_n))
        print('Coherent drive: amp = {}, length = {} ns'.format(cav_amp, cav_len))

        nx, ny = np.shape(df)
                
        """Renaming the columns of repeated pi pulses"""
        l = []
        for i in range(ny):
            l.append('π%i'%i)
        df.columns = l

        df['n'] = num

        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols].sort_values(by=['n'])

        """Find out the unique Fock levels and their occurences"""

        (unique, counts) = np.unique(df['n'], return_counts=True)

        print(unique, counts)
        
        p_m_counts = []
        p_n_counts = []
        
        index = 0
        for ii in range(len(unique)):
            pm_temp = []
            pn_temp = []
            
            fstate_in = unique[ii]
            """T and E matrices for the state prep part"""
            if fstate_in ==0:
                cavity_t1 = self.cavity_t1/(fstate_in+1)
                Pnm =  self.cavity_nth * (1-np.exp(-self.trigger_period/cavity_t1))
            else: 
                cavity_t1 = self.cavity_t1/(fstate_in)
                Pnm =  (1-np.exp(-self.trigger_period/cavity_t1)) + self.cavity_nth * (1-np.exp(-self.trigger_period/cavity_t1))
                
            Pmn = 0 + 0 #assuming that the population at (n+1) is negligible and (n-1) we will estimate\
            Pge = self.qubit_nth * (1-np.exp(-self.trigger_period/self.qubit_t1)) +\
                (1-np.exp(-self.pi_length/self.qubit_t2))
            Peg = (1-np.exp(-self.trigger_period/self.qubit_t1)) + \
                (1-np.exp(-self.pi_length/self.qubit_t2))

            T_m = asarray([[(1-Pmn)*(1-Pge), (1-Pmn)*Pge, Pmn*Pge, Pmn*(1-Pge)],
                 [(1-Pmn)*Peg, (1-Pmn)*(1-Peg), Pmn*(1-Peg), Pmn*Peg],
                 [Pnm*(1-Pge), Pnm*Pge, (1-Pnm)*Pge, (1-Pnm)*(1-Pge)],
                 [Pnm*Peg, Pnm*(1-Peg), (1-Pnm)*(1-Peg), (1-Pnm)*Peg]])

            E_m = 0.5*asarray([[1-g_infidelity, g_infidelity],
                [e_infidelity, 1- e_infidelity],
                [1-g_infidelity, g_infidelity],
                [e_infidelity, 1- e_infidelity]])     

            """T and E matrices for the second half of the experiment"""
            cavity_t1 = self.cavity_t1/(fstate_in+1)
            Pnm =  (1-np.exp(-self.trigger_period/cavity_t1)) + self.cavity_nth * (1-np.exp(-self.trigger_period/cavity_t1))

            Pmn = 0 + 0 #assuming that the population at (n+1) is negligible and (n-1) we will estimate\
            Pge = self.qubit_nth * (1-np.exp(-self.trigger_period/self.qubit_t1)) +\
                (1-np.exp(-self.pi_length/self.qubit_t2))
            Peg = (1-np.exp(-self.trigger_period/self.qubit_t1)) + \
                (1-np.exp(-self.pi_length/self.qubit_t2))

            T_n = asarray([[(1-Pmn)*(1-Pge), (1-Pmn)*Pge, Pmn*Pge, Pmn*(1-Pge)],
                 [(1-Pmn)*Peg, (1-Pmn)*(1-Peg), Pmn*(1-Peg), Pmn*Peg],
                 [Pnm*(1-Pge), Pnm*Pge, (1-Pnm)*Pge, (1-Pnm)*(1-Pge)],
                 [Pnm*Peg, Pnm*(1-Peg), (1-Pnm)*(1-Peg), (1-Pnm)*Peg]])

            E_n = 0.5*asarray([[1-g_infidelity, g_infidelity],
                [e_infidelity, 1- e_infidelity],
                [1-g_infidelity, g_infidelity],
                [e_infidelity, 1- e_infidelity]])
            
            for jj in range(counts[ii]):
                """State preparation probabilities at the end or at the beginning of m π pulses"""
                meas_seq = df.iloc[index + jj][1:1+npi_m]
                gamma_matrix = self.gamma(meas_seq, T_m, E_m)
                if at_end==True: #Probablitity of state surviving till the end
                    P0_last = gamma_matrix[-1,0] + gamma_matrix[-1,1]
                    P1_last = gamma_matrix[-1,2] + gamma_matrix[-1,3]
                else:#Probability at the beginnning
                    P0_last = gamma_matrix[0,0] + gamma_matrix[0,1]
                    P1_last = gamma_matrix[0,2] + gamma_matrix[0,3]   
                pm_temp.append(P1_last/P0_last)          
                
                """After the coherent drive, probabilities at the begining of n π pulses"""    
                meas_seq = df.iloc[index + jj][npi_m:]
                gamma_matrix = self.gamma(meas_seq, T_n, E_n)
                P0_first = gamma_matrix[0,0] + gamma_matrix[0,1]
                P1_first = gamma_matrix[0,2] + gamma_matrix[0,3]   
                pn_temp.append(P1_first/P0_first)

            index = index + counts[ii]
            p_m_counts.append(pm_temp)
            p_n_counts.append(pn_temp)

        return alpha, unique, counts, p_m_counts, p_n_counts

# Computing the fidelity based on HMM

## n=0

In [78]:
qubit_params = {'t1':100, 't2':136, 'nth':1e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n0_bd_fidelity'
fock_state = 0
filelist = [fock_state]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=True)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00000_n0_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [9635   98  175   92]
(9635, 2)
6605
0
0 0.0


In [18]:
6605/9635

0.6855215360664245

In [79]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/9635)

9255
0.9605604566683965


### n+1

In [41]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n0_bd_fidelity'
fock_state = 0
filelist = [1]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00001_n0_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [9903   29   44   24]
(9903, 2)
0
0
0 nan


<ipython-input-41-33971c46e95f>:40: RuntimeWarning: invalid value encountered in long_scalars
  print(count_p, count_p/count_m)


In [42]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/9903)

0
0.0


## n=1

In [62]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n1_bd_fidelity'
fock_state = 1
filelist = [0]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00000_n1_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [ 748 7779  637  836]
(7779, 2)
4716
0
0 0.0


In [63]:
th1 = 1e6
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/7779)

126
0.01619745468569225


### n+1

In [67]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n1_bd_fidelity'
fock_state = 1
filelist = [1]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=True)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00001_n1_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [ 881 7652  703  764]
(7652, 2)
0
0
0 nan


<ipython-input-67-615d9fc729bc>:40: RuntimeWarning: invalid value encountered in long_scalars
  print(count_p, count_p/count_m)


In [66]:
th1 = 1e6
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/7652)

1
0.00013068478829064296


In [75]:
th1 = 1e-6
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/7652)

7652
1.0


### n-1

In [51]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n1_bd_fidelity'
fock_state = 1
filelist = [2]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00002_n1_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [ 921 7659  695  725]
(7659, 2)
146
0
0 0.0


In [52]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/7659)

353
0.04608956782869826


## n=2

In [37]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n2_bd_fidelity'
fock_state = 2
filelist = [0]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00000_n2_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [1932 1275 5539 1254]
(5539, 2)
2552
0
0 0.0


In [38]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/5539)

4065
0.7338869832099657


### n+1

In [47]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n2_bd_fidelity'
fock_state = 2
filelist = [1]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00001_n2_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [2054 1234 5509 1203]
(5509, 2)
1
0
0 0.0


In [48]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/5509)

3
0.0005445634416409512


### n-1

In [54]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n2_bd_fidelity'
fock_state = 2
filelist = [2]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00002_n2_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [2101 1256 5404 1239]
(5404, 2)
235
0
0 0.0


In [55]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/5404)

489
0.09048852701702442


## n=3

In [39]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n3_bd_fidelity'
fock_state = 3
filelist = [0]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00000_n3_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [3965 2331 1079 2625]
(2625, 2)
873
0
0 0.0


In [40]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/2625)

1567
0.5969523809523809


### n+1

In [58]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n3_bd_fidelity'
fock_state = 3
filelist = [2]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

..\data\00002_n3_bd_fidelity.h5
# of π at m= 10, at n = 5
Coherent drive: amp = 0.0, length = 400 ns
[0 1 2 3] [3951 2393 1061 2595]
(2595, 2)
0
0
0 nan


<ipython-input-58-c139f637f232>:40: RuntimeWarning: invalid value encountered in long_scalars
  print(count_p, count_p/count_m)


In [61]:
th1 = 1e2
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/2595)

1
0.0003853564547206166


### n-1

In [ ]:
qubit_params = {'t1':100, 't2':136, 'nth':5e-2}
cavity_params = {'t1':650, 'nth':0.001}
readout_params = {'length':3.2, 'trigger':7.2, 'pi_pulse':3}

expt_name = 'n3_bd_fidelity'
fock_state = 3
filelist = [1]
data_path = '..//data//'

for ii, i in enumerate(filelist):
    filename = "..\\data\\" + str(i).zfill(5) + "_"+expt_name+".h5"

    p_m_counts = []
    p_n_counts = []
    alphas = []

    print(filename)
    obj = hmm_analysis(qubit_params=qubit_params, cavity_params=cavity_params, readout_params=readout_params)
    alpha, unique, counts, p_m, p_n = obj.stateprep(data_filename=filename, at_end=False)
    p_m_counts.extend(p_m[fock_state])
    p_n_counts.extend(p_n[fock_state])        
    alphas.append(alpha)

df = pd.DataFrame(p_m_counts)

df.columns = ['p_m']
df['p_n'] = p_n_counts

print(np.shape(df))
th1 = 1e5
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)
count_n = df['p_n'][df['p_n']>th2].count()
print(count_n)

count_p = df['p_n'][(df['p_m']>th1) & (df['p_n']>th2)].count()

print(count_p, count_p/count_m)

In [ ]:
th1 = 1e3
th2 = 1e8

count_m = df['p_m'][df['p_m']>th1].count()
print(count_m)

print(count_m/2625)